### Mount (montar) a Azure Data Lake Storage para el projecto
1. Obtener el valor del client_id, tenant_id y client_secret del key Vault
2. Configurar Spark con APP/Client id , Directory/tenant Id & Secret
3. Usar el método "mount" de "utility" para montar el almacenamiento.
4. Explorar otras utilidades del sistema de archivos relacionados con el montaje (list all mounts, unmounts)


In [0]:
def mount_adls(storage_account_name, container_name):
    # Obnetner secret key de key vault
    client_id = dbutils.secrets.get(scope="movie-historyl-secret-scope",
                    key="client2-id")
    tenant_id = dbutils.secrets.get(scope="movie-historyl-secret-scope",
                    key="tenant2-id")
    client_secret = dbutils.secrets.get(scope="movie-historyl-secret-scope", key="client-secret")
    
    # Establecer configuarciones de Spark
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Evaluar  si el montaje ya existe y de ser asi desmontar
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    # Mount al contenedor del storage account
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)
    # Listar los Mounts
    display(dbutils.fs.mounts())

# def unmount_adls(storage_account_name, container_name):
#     dbutils.fs.unmount(f"mnt/{storage_account_name}/{container_name}")
#     display(dbutils.fs.mounts())


#### Montar el contenedor "bronce"

In [0]:
mount_adls("moviehistoryl", "bronce")

%md
#### Montar el contenedor "silver"

In [0]:
mount_adls("moviehistoryl", "silver")

%md
%md
#### Montar el contenedor "gold"

In [0]:
mount_adls("moviehistoryl", "gold")

In [0]:
meta = dbutils.secrets.list(scope="movie-historyl-secret-scope")
secrets_names = [n.key for n in meta ]
print(secrets_names)

In [0]:
dbutils.fs.unmount("/mnt/moviehistoryl/demo")